# 使用 Nsight Compute 优化在线 5G 机器学习

## 02 运行 APSM 代码

我们首先构建本教程中使用的 APSM 代码。在代码编译过程中，我们可以了解它的含义。

该示例基于 cmake。运行以下命令来构建应用程序。在构建代码时，我们先来看一下有关 APSM 的简短介绍。有关完整详细信息，请参阅此 [演示文稿](/lab/files/ncu/assets/s9922-softwareization-of-mobile-radio-networks.pdf)。

In [ ]:
!mkdir /dli/task/ncu/apsm/cpp/build
%cd /dli/task/ncu/apsm/cpp/build
!cmake ..
!make clean && make -j

`自适应投影次梯度法（APSM）` 是一种无线通信代码。在本实验中，通信系统负责比特和符号与模拟信号之间的转换，这些模拟信号被放大并通过天线无线传输。接收端必须执行逆过程：接收信号、处理信号、`检测符号` 并将它们映射回比特。无线信道建模很困难，因为它有损耗、噪声、干扰，而且动态性很强。

<img src="images/apsm_01.png">

我们在本例中针对的算法使用四相移键控 (QPSK) 调制方案。我们将两个比特编码到每个符号中，因此我们得到四个可能的符号（来自组合 00、01、10、11）。

<img src="images/apsm_02.png" width="700">

在左侧，我们为两个用户中的每一个发送了符号。在右侧，我们可以看到经过信道后收到的内容。我们可以看到接收的样本如何形成与传输的模式相似的模式，但更小、旋转且“更模糊”，因为每个点都有些偏离中心。随着我们发送越来越多的符号（我们可以看到 16 个粗略位置），它们会在某个中心周围形成一个云。

我们希望能够将每个接收的样本（右侧的点）映射（分配）到传输的符号之一（左侧的四个点）。并且我们希望能够实时、低延迟地执行此操作，适用于任何合理的信道条件。这通常是通过测量已知模式并使用它们来提取用于表征信道的参数来实现的。

“这就像两个人同时在房间里说话。我们如何区分每个人说的话？”为了解决这个问题，可以添加更多天线。回到房间类比，这相当于转动头部或靠近一步，以改善特定说话者相对于其他说话者的聆听效果。

有一些方法可以从多个天线转换为原始符号，但它们面临挑战。“这就是机器学习可以提供帮助的地方。我们不是对信道进行建模，而是对其进行学习，从而隐式地创建模型。”它承担了将接收到的样本有效地映射到传输的符号的任务。

单击下方 `...` 以进一步阅读。

这是另一个示例，但有多个天线。由于天线不在同一位置，每个天线接收的信号略有不同。请注意每个图表的旋转方式不同。

<img src="images/apsm_04.png" width="700">

APSM 通过将样本投影到一组学习过的表面来近似符号位置。这些表面是从训练数据中学习到的。该方法实现的是基于已知训练集 `r,b` 的函数 `f` 的函数近似 `f*`，作为某些函数的加权贡献的总和，这样我们就可以最小化误差值。这是通过迭代并行投影实现的。

<img src="images/apsm_05.png" width="700">

使用 GPU 来加速这种算法特别合适。我们刚刚描述的 APSM 投影可以由 GPU 并行且非常高效地完成。

使用单个 GPU，我们可以在短短 1.5 毫秒内估算符号，并且可以通过一组导频（导频是参考和同步信号）在几毫秒的范围内训练模型。作为参考，5G NR 帧为 10 毫秒，子帧为 1 毫秒。这意味着我们有一个软件解决方案可以并行进行投影、并行处理样本并并行运行缩减。

现在编译应该已经完成了。执行以下命令从 cpp/build 运行代码：

In [ ]:
%cd /dli/task/ncu/apsm/cpp/build
!bin/APSM_tool -m QAM16 -s ../data/offline/rx/time/rxData_QAM16_alltx_converted.bin -r ../data/offline/tx/NOMA_signals_qam16_complex.bin

输出 `shows a command line representation of the detected symbol errors` 。如果此设置中第一个用户的错误率为 `0.05 %` ，则它按预期工作。（该算法同时检测多个用户的符号。）

让我们看看 [步骤 03](03_nsight_systems.ipynb) 中应用程序的时间线